# 🏆 Cantina Chatbot: An Intelligent Food Query Assistant  

## 🍽️ Introduction  

Cantina Chatbot is a smart, interactive assistant designed to help users with menu-related queries. It leverages **Natural Language Processing (NLP)** techniques to understand user messages, extract relevant food-related information, and provide accurate responses.  

### 🔥 Key Features:  
- **Spell Correction**: Automatically corrects user typos using **TextBlob**.  
- **Named Entity Recognition (NER)**: Uses **spaCy** to identify food items and products from user messages.  
- **Context Awareness**: Remembers the last detected food-related entity for better continuity in conversation.  
- **Intent Matching**: Finds the most relevant response using **word similarity analysis** with **spaCy** word vectors.  
- **Diverse Responses**: Dynamically generates responses based on detected entities to make the conversation feel more natural.  
- **Exit Confirmation**: Ensures users do not accidentally exit the chat by requiring confirmation.  

The chatbot is built using **Python** and libraries such as `spaCy`, `NLTK`, and `TextBlob`, making it both powerful and efficient for handling customer inquiries in a restaurant or online ordering system.  

---

In [16]:
# Download necessary NLTK resources
#  nltk.download("stopwords")
#  nltk.download("punkt")
#  nltk.download("averaged_perceptron_tagger")
# !pip install spacy
# !python -m spacy download en_core_web_md
# !pip install textblob
# !python -m textblob.download_corpora

In [21]:
import re
import spacy
import random
import nltk
from collections import Counter
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

# Load language model
# nlp = spacy.load("en_core_web_sm") 
nlp = spacy.load("en_core_web_md")  # We also can use "en_core_web_lg" para más precisión, pero ralentiza.
stop_words = set(stopwords.words("english"))

# Exit commands
exit_commands = ("quit", "goodbye", "exit", "no")

# Dynamic responses for variety
response_templates = [
    "Yes! The {} has a gluten-free option, but it is not vegan.",
    "We offer many options with the {}, such as mashed potatoes and steamed vegetables, or meat if you liked it",
    "Our {} contains habanero, so it has a spicy kick!"
]
fallback_responses = [
    "I'm not sure about that. Could you rephrase your question?",
    "Hmm, I don’t know. Can you ask in a different way?",
    "I didn’t understand. Could you clarify?"
]

class ChatBot:
    """An enhanced chatbot that understands food-related queries better."""

    def __init__(self):
        self.context = None  # Stores the last recognized entity

    def make_exit(self, user_message): # Confirms before exiting the conversation
        if any(exit_command in user_message.lower() for exit_command in exit_commands):
            confirmation = input("Are you sure you want to exit? (yes/no) ").strip().lower()
            return confirmation == "yes"
        return False

    def preprocess(self, input_sentence): # Lowercases, removes punctuation, tokenizes, and removes stopwords.
        input_sentence = input_sentence.lower()
        input_sentence = re.sub(r"[^\w\s]", "", input_sentence)  # Remove punctuation
        tokens = word_tokenize(input_sentence)
        return [i for i in tokens if i not in stop_words]

    def correct_spelling(self, user_message): # Uses TextBlob to correct typos.
        return str(TextBlob(user_message).correct())

    def extract_entities(self, user_message): # Uses spaCy's Named Entity Recognition (NER) to extract food-related entities
        doc = nlp(user_message)
        food_items = [ent.text for ent in doc.ents if ent.label_ in ["FOOD", "PRODUCT"]]
        
        if food_items:
            self.context = food_items[-1]  # Remember the last recognized entity
            return food_items[-1]
        
        # If no entity is found, try using nouns as a fallback
        tagged_user_message = pos_tag(self.preprocess(user_message))
        message_nouns = [word for word, tag in tagged_user_message if tag.startswith("N")]
        
        if message_nouns:
            self.context = message_nouns[-1]
            return message_nouns[-1]

        return self.context if self.context else "food"

    def find_intent_match(self, responses, user_message): #Uses spaCy similarity to find the best matching response.
        user_doc = nlp(user_message)
        response_scores = [(resp, user_doc.similarity(nlp(resp))) for resp in responses]
        best_response = max(response_scores, key=lambda x: x[1])[0]

        return best_response if max(response_scores, key=lambda x: x[1])[1] > 0.3 else random.choice(fallback_responses)

    def chat(self): # Main chat loop for interacting with the user.
        print("👋 Hey! What would you like to know about our menu today?")
        
        while True:
            try:
                user_message = input("> ").strip()
                user_message = self.correct_spelling(user_message)  # Auto-correct typos
                
                if self.make_exit(user_message):
                    print("Goodbye! Have a great day! 😊")
                    break

                response = self.respond(user_message)
                print(response)

            except Exception as e:
                print("Oops! Something went wrong. Try again.")

    def respond(self, user_message): # Generates a response based on user input.
        best_response = self.find_intent_match(response_templates, user_message)
        entity = self.extract_entities(user_message)
        return best_response.format(entity)


# Initialize and start chatbot
bot = ChatBot()
bot.chat()


👋 Hey! What would you like to know about our menu today?


>  any meal?


We offer many options with the meal, such as mashed potatoes and steamed vegetables, or meat if you liked it


>  and do you have spicy?


Yes! The spice has a gluten-free option, but it is not vegan.


>  ok but later, i need to go now. Bye
Are you sure you want to exit? (yes/no)  yes


Goodbye! Have a great day! 😊


# 🎯 Conclusion and Future Improvements  

## ✅ Strengths:  
Cantina Chatbot effectively handles **menu-related questions** and **corrects spelling errors**, making it user-friendly. It also utilizes **machine learning-based NLP techniques** to enhance response accuracy.  

## 🔧 Areas for Improvement:  
- **Improve Entity Recognition**: Expand food-related entity detection using a custom **spaCy pipeline** trained on restaurant-specific data.  
- **Enhance Small Talk Handling**: Integrate a conversational model (e.g., **GPT-based** responses) to make interactions more engaging.  
- **Implement Memory for Conversations**: Store user preferences or past queries for **personalized recommendations**.  
- **Add More Custom Responses**: Increase the variety of dynamic responses to avoid repetition.  
- **Deploy as a Web App**: Integrate with a **Flask** or **FastAPI** backend for real-world usage in food ordering platforms.  

*In the comming day we will work for this last  imporvement, that is, workin on the Deployment, so Cantina Chatbot can evolve into a **fully interactive virtual assistant** for restaurants, catering businesses, and food delivery services.* 🚀